<a href="https://colab.research.google.com/github/Wyatt-Kugler/Cartpole-RL-Model/blob/main/Cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium
!pip install imageio
!pip install IPython
!pip install numpy
!pip install tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_folder = "/content/drive/MyDrive/Cartpole/Cartpole_GIFs/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gymnasium as gym
import imageio
import numpy as np

from IPython.display import Image
from collections import defaultdict

In [ ]:
import numpy as np
from collections import defaultdict

def discretize(obs, bins):
    """Convert continuous obs to discrete indices."""
    upper_bounds = [4.8, 5, 0.418, 5]  # CartPole-v1 env limits
    lower_bounds = [-4.8, -5, -0.418, -5]

    ratios = [(obs[i] - lower_bounds[i]) / (upper_bounds[i] - lower_bounds[i]) for i in range(4)]
    new_obs = [int(np.clip(r * (bins[i]-1), 0, bins[i]-1)) for i, r in enumerate(ratios)]
    return tuple(new_obs)

class CartAgent:
    def __init__(
        self,
        env,
        learning_rate,
        initial_epsilon,
        epsilon_decay,
        final_epsilon,
        discount_factor=0.95,
    ):
        """ Initializes Q-Learning Agent """
        self.env = env
        self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))
        self.lr = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon
        self.training_error = []

    def get_action(self, obs):
        """ Choose action using epsilon-greedy policy """
        state_index = discretize(obs, bins=[10, 10, 10, 10])
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.q_values[state_index])

    def update(self, obs, action, reward, terminated, next_obs):
        """ Update Q-values """
        obs_index = discretize(obs, bins=[10, 10, 10, 10])
        next_index = discretize(next_obs, bins=[10, 10, 10, 10])

        # only include future reward if episode is not terminated
        future_q_value = np.max(self.q_values[next_index]) * (not terminated)
        target = reward + self.discount_factor * future_q_value

        td_error = target - self.q_values[obs_index][action]
        self.q_values[obs_index][action] += self.lr * td_error

        self.training_error.append(td_error)

    def decay_epsilon(self):
        """ Decay exploration rate """
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)


In [ ]:
learning_rate = 0.01
n_episodes = 5000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)
final_epsilon = 0.1

env = gym.make("CartPole-v1", render_mode="rgb_array", max_episode_steps=1000)

agent = CartAgent(
    env=env,
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
)


In [ ]:
from tqdm import tqdm
import imageio

Recording_Number = 0
gif_files = []

for episode in tqdm(range(n_episodes)):
  recording = False
  if episode % 100 == 0:
    frames = []
    Recording_Number += 1
    recording = True
  obs, info = env.reset()

  if recording:
    frames.append(env.render())

  done = False

  while not done:
    action = agent.get_action(obs)
    next_obs, reward, terminated, truncated, info = env.step(action)

    agent.update(obs, action, reward, terminated, next_obs)

    done = terminated or truncated

    if recording:
      frames.append(env.render())
    obs = next_obs
  if recording:
    gif_path = f"cartpole_loop_{Recording_Number}.gif"

    full_path = save_folder + gif_path

    imageio.mimsave(full_path, frames, fps=30)

    gif_files.append(full_path)

  agent.decay_epsilon()

  0%|          | 0/5000 [00:00<?, ?it/s]

Saving GIF to: /content/drive/MyDrive/Cartpole/Cartpole_GIFs/cartpole_loop_1.gif


  2%|▏         | 95/5000 [00:03<01:12, 67.39it/s]

Saving GIF to: /content/drive/MyDrive/Cartpole/Cartpole_GIFs/cartpole_loop_2.gif


  4%|▍         | 200/5000 [00:09<00:57, 83.76it/s]

...

 96%|█████████▌| 4792/5000 [01:22<00:02, 85.24it/s]

Saving GIF to: /content/drive/MyDrive/Cartpole/Cartpole_GIFs/cartpole_loop_49.gif


 98%|█████████▊| 4895/5000 [01:24<00:01, 74.97it/s]

Saving GIF to: /content/drive/MyDrive/Cartpole/Cartpole_GIFs/cartpole_loop_50.gif


100%|██████████| 5000/5000 [01:26<00:00, 57.77it/s]
